## 问题描述

使用 PINN 求解给定几何结构和流动参数的参数化流体流动问题

模拟相同的切屑流动，但现在在单次参数化运行中改变切屑高度和宽度。

<img src="chip_2d_parameterized.png" alt="Drawing" style="width: 800px;"/>

## 练习

本题的主要目标是使用 PINN 正确地公式化问题。为了实现这一点，你必须成功完成以下部分：
1. 为问题定义正确的*参数化*几何结构
2. 设置正确的边界条件和方程
3. 创建神经网络并求解*参数化*问题

成功完成本题后，应该能够生成一个可以在我们选择的任何几何参数组合下进行推理的网络。例如，该模型应该能够为切屑高度为 0.4 且宽度为 1.4 的几何结构创建绘图。示例如下。记住，一旦模型训练完成，你就可以推断任何几何组合，而无需再次求解该问题。是不是很酷？

<img src="challenge_results_param_updated.png" alt="Drawing" style="width: 800px;"/>

在此模板中，我们为您提供了一个框架代码，您需要填写该代码来定义和解决参数化问题。如果您已成功完成练习 1，那么转到此部分应该相当容易，因为只需进行一些小的修改。让我们从导入所需的软件包开始。

**注意：您需要编辑位于 `chip_2d/` 目录中的 [`chip_2d_parameterized_template.py`](../../source_code/chip_2d/chip_2d_parameterized_template.py) 脚本。**

```python
import numpy as np
from sympy import Symbol, Eq

import modulus
from modulus.hydra import to_absolute_path, ModulusConfig, instantiate_arch
from modulus.utils.io import csv_to_dict
from modulus.solver import Solver
from modulus.domain import Domain
from modulus.geometry.primitives_2d import Rectangle, Line, Channel2D
from modulus.utils.sympy.functions import parabola
from modulus.eq.pdes.navier_stokes import NavierStokes
from modulus.eq.pdes.basic import NormalDotVec
from modulus.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint,
    IntegralBoundaryConstraint,
)

from modulus.domain.validator import PointwiseValidator
from modulus.key import Key
from modulus.node import Node
```

现在我们已经定义了模拟参数和符号变量，用于参数化下面的几何结构。如之前的教程所示，我们定义了训练期间数据点采样的范围。您可以随意尝试不同的变量范围。

```python
@modulus.main(config_path="conf", config_name="config_param")
def run(cfg: ModulusConfig) -> None:

    # make list of nodes to unroll graph on
    ns = NavierStokes(nu=0.02, rho=1.0, dim=2, time=False)
    normal_dot_vel = NormalDotVec(["u", "v"])
    flow_net = instantiate_arch(
        input_keys=[placeholder],
        output_keys=[placeholder],
        cfg=cfg.arch.fourier_net,
    )
    nodes = (
        ns.make_nodes()
        + normal_dot_vel.make_nodes()
        + [flow_net.make_node(name="flow_network", jit=cfg.jit)]
    )

    # add constraints to solver
    # simulation params
    channel_length = (-2.5, 2.5)
    channel_width = (-0.5, 0.5)
    chip_pos = -1.0
    inlet_vel = 1.5

    # paramteric variables
    chip_height = Symbol("chip_height")  # Not fixed anymore
    chip_width = Symbol("chip_width")    # Not fixed anymore
    chip_height_range = (0.4, 0.8)
    chip_width_range = (0.6, 1.4)
    param_ranges = {chip_height: chip_height_range, chip_width: chip_width_range}

    # define sympy variables to parametrize domain curves
    x, y = Symbol("x"), Symbol("y")

    # define geometry
    channel = placeholder
    inlet = placeholder
    outlet = placeholder
    rec = placeholder
    flow_rec = placeholder
    geo = channel - rec
    geo_hr = geo & flow_rec
    geo_lr = geo - flow_rec
    x_pos = Symbol("x_pos")
    integral_line = Line((x_pos, channel_width[0]), (x_pos, channel_width[1]), 1)
    x_pos_range = {
        x_pos: lambda batch_size: np.full(
            (batch_size, 1), np.random.uniform(channel_length[0], channel_length[1])
        )
    }
```

现在完成下面的约束定义。请记住，这次您必须将参数键及其范围的字典添加为 `param_ranges` 参数的输入。入口边界条件的示例已显示。

*注意：对于积分连续性平面，您有两个字典。一个用于随机采样的* `x_pos` *，另一个用于参数变量本身。确保将这两个字典都作为* `param_ranges` *的输入。*

```python
    #TODO: Replace all the placeholders with appropriate values 

    # make domain
    domain = Domain()

    # inlet
    inlet_parabola = parabola(y, channel_width[0], channel_width[1], inlet_vel)
    inlet = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=inlet,
        outvar={"u": inlet_parabola, "v": 0},
        batch_size=cfg.batch_size.inlet,
        parameterization=param_ranges,
    )
    domain.add_constraint(inlet, "inlet")

    # outlet
    outlet = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=outlet,
        outvar=placeholder,
        batch_size=cfg.batch_size.outlet,
        criteria=Eq(x, channel_length[1]),
        parameterization=placeholder,
    )
    domain.add_constraint(outlet, "outlet")

    # no slip
    no_slip = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=geo,
        outvar=placeholder,
        batch_size=cfg.batch_size.no_slip,
        parameterization=placeholder,
    )
    domain.add_constraint(no_slip, "no_slip")

    # interior lr
    interior_lr = PointwiseInteriorConstraint(
        nodes=nodes,
        geometry=geo_lr,
        outvar=placeholder,
        batch_size=cfg.batch_size.interior_lr,
        bounds={x: channel_length, y: channel_width},
        lambda_weighting=placeholder,
        parameterization=placeholder,
    )
    domain.add_constraint(interior_lr, "interior_lr")

    # interior hr
    interior_hr = PointwiseInteriorConstraint(
        nodes=nodes,
        geometry=geo_hr,
        outvar=placeholder,
        batch_size=cfg.batch_size.interior_hr,
        bounds={x: channel_length, y: channel_width},
        lambda_weighting=placeholder,
        parameterization=placeholder,
    )
    domain.add_constraint(interior_hr, "interior_hr")

    # integral continuity
    def integral_criteria(invar, params):
        sdf = geo.sdf(invar, params)
        return np.greater(sdf["sdf"], 0)

    integral_continuity = IntegralBoundaryConstraint(
        nodes=nodes,
        geometry=integral_line,
        outvar=placeholder,
        batch_size=cfg.batch_size.num_integral_continuity,
        integral_batch_size=cfg.batch_size.integral_continuity,
        lambda_weighting=placeholder,
        criteria=geo.sdf > 0,
        parameterization=placeholder,
    )
    domain.add_constraint(integral_continuity, "integral_continuity")
```

现在，向问题中添加验证数据。与教程示例类似，针对单个配置验证此参数化解决方案。请记住，此问题的网络有四个输入（x、y、chip_height 和 chip_width）。因此，您的验证数据也必须具有 CSV 文件中每个点的对应输入。

```python
    # add validation data
    mapping = {"Points:0": "x", "Points:1": "y", "U:0": "u", "U:1": "v", "p": "p"}
    openfoam_var = csv_to_dict(to_absolute_path("openfoam/2D_chip_fluid0.csv"), mapping)
    openfoam_var["x"] -= 2.5  # normalize pos
    openfoam_var["y"] -= 0.5
    
    # TODO: Add the arrays for 'chip_height' and 'chip_width'
    
    openfoam_invar_numpy = {
        key: value
        for key, value in openfoam_var.items()
        if key in ["x", "y", "chip_height", "chip_width"]
    }
    openfoam_outvar_numpy = {
        key: value for key, value in openfoam_var.items() if key in ["u", "v", "p"]
    }
    openfoam_validator = PointwiseValidator(
        invar=openfoam_invar_numpy, true_outvar=openfoam_outvar_numpy, nodes=nodes
    )
    domain.add_validator(openfoam_validator)
```

正如开头所讨论的，我们希望无需反复训练即可实现多种设计的可视化。您可以定义一个 `PointwiseInferencer` 来实现这一点。只需更改此域中的值，并以 `eval` 模式执行脚本，即可使用已训练好的网络实现多种设计的可视化。

```python
    inference = PointwiseInferencer(
        openfoam_invar_numpy,     # TODO: The invar array can be changed to infer at different geometry combinations
        ["u", "v", "p"],
        nodes,
        batch_size=1024,
    )
    domain.add_inferencer(inference, "inf_data")
```

现在，通过定义求解器来解决我们的参数问题来完成这个问题。该问题的重要超参数已在配置文件中定义。您可以随意调整它们，并观察其对结果和收敛速度的影响。

```python
    # make solver
    slv = Solver(cfg, domain)

    # start solver
    slv.solve()


if __name__ == "__main__":
    run()
```